In [1]:
import os
import time
import copy
import numpy as np

from PIL import Image
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [2]:
torch.__version__, torchvision.__version__

('1.6.0', '0.7.0')

In [3]:
def load_img(path):
    img = Image.open(path)
    img = img.convert("RGB")
    return img

In [4]:
train_transforms_list = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.RandomAffine(degrees=(-20, 20), translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms_list = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_ds = datasets.DatasetFolder(
    "/Users/roman.ushakov/Desktop/clouds_dataset/train/", 
    loader=load_img,
    extensions=("jpg", "png"),
    transform=train_transforms_list,
)

val_ds = datasets.DatasetFolder(
    "/Users/roman.ushakov/Desktop/clouds_dataset/val/", 
    loader=load_img,
    extensions=("jpg", "png"),
    transform=val_transforms_list,
)

In [5]:
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=4, shuffle=True, num_workers=0)

In [6]:
model = torchvision.models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 2)
params_to_update = model.parameters()
optimizer = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
device = "cpu"

In [7]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [8]:
image_dataloaders = {"train": train_loader, "val": val_loader}

In [9]:
best_model, hist = train_model(model, image_dataloaders, criterion, optimizer, num_epochs=4, is_inception=False)

Epoch 0/3
----------
train Loss: 0.4218 Acc: 0.8005
val Loss: 0.3467 Acc: 0.8857

Epoch 1/3
----------
train Loss: 0.5340 Acc: 0.7896
val Loss: 0.1286 Acc: 0.9571

Epoch 2/3
----------
train Loss: 0.4009 Acc: 0.8470
val Loss: 0.1049 Acc: 0.9286

Epoch 3/3
----------
train Loss: 0.3404 Acc: 0.8825
val Loss: 0.1112 Acc: 0.9286

Training complete in 8m 58s
Best val Acc: 0.957143


In [10]:
torch.save(best_model.state_dict(), "best.pt")